In [243]:
# Definimos el directorio 

from pathlib import Path
import os

# Obtenemos la ruta del notebook actual
notebook_dir = Path().resolve()
print("Directorio actual:", notebook_dir)

# Cambia el directorio de trabajo a esa ruta
os.chdir(r"C:\Users\admin\Documents\Códigos Pytohn\UNIDAD 3")

# Confirmamos el cambio
print("Nuevo directorio de trabajo:", os.getcwd())

Directorio actual: C:\Users\admin
Nuevo directorio de trabajo: C:\Users\admin\Documents\Códigos Pytohn\UNIDAD 3


In [323]:
# === Librerías ===
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import requests, zipfile, io
import statsmodels.api as sm

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures 
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
from scipy import stats

# Configuración general
pd.set_option('display.max_columns', None)
pd.options.display.float_format = '{:,.2f}'.format
sns.set(style="whitegrid", palette="deep")

In [241]:
# === Cargar base del TP2 ===
import os
import pandas as pd

# Ruta actualizada a la carpeta del TP2
DATOS = r"C:\Users\admin\Documents\Códigos Pytohn\UNIDAD 3"
ruta = os.path.join(DATOS, "eph_base.csv")

# Cargar base respondieron
eph = pd.read_csv(ruta)

# Verificación rápida
print("Base cargada correctamente.")
print("Filas y columnas:", eph.shape)
print("\nVista previa:")
display(eph.head(3))

Base cargada correctamente.
Filas y columnas: (7340, 37)

Vista previa:


,componente,ano4,codusu,nro_hogar,Sexo,ch06,ch07,ch08,nivel_ed,estado,cat_ocup,cat_inac,itf,ipcf,pp07h,pp03c,p47t,p21,pp03g,Edad,adulto_equiv,ad_equiv_hogar,ingreso_necesario,pobre,edad2,ch10,ch12,ch13,ch14,educ,itf_2025,linea_pobreza,pp3e_tot,pp3f_tot,ch03,horastrab,ix_tot
0,1.00,2005,125814,1,Varón,46,Casado,Obra social (incluye PAMI),Secundaria Incompleta,Ocupado,Obrero o empleado,NaN,"2,400.00",480.00,Sí,...un sólo empleo/ocupación/actividad?,"2,400.00","2,400.00",No,46 a 60 años,1.00,3.93,805.93,0,2116,2,4,2,2.00,10.00,"2,518,778.11","845,811.04",48.00,0.00,1,48.00,5.00
1,2.00,2005,125814,1,Mujer,32,Casado,Obra social (incluye PAMI),Secundaria Incompleta,Inactivo,NaN,Ama de casa,"2,400.00",480.00,NaN,NaN,0.00,0.00,NaN,30 a 45 años,0.77,3.93,805.93,0,1024,2,4,2,2.00,10.00,"2,518,778.11","845,811.04",0.00,0.00,2,0.00,5.00
2,3.00,2005,125814,1,Varón,14,Soltero,Obra social (incluye PAMI),Primaria Completa,Inactivo,NaN,Estudiante,"2,400.00",480.00,NaN,NaN,0.00,0.00,NaN,14 años,0.96,3.93,805.93,0,196,1,3,2,7.00,8.00,"2,518,778.11","845,811.04",0.00,0.00,3,0.00,5.00


In [163]:
# Hay duplicados?
print("Duplicados:", eph.duplicated().sum())

# Hay valores faltantes?
print("\n Missings:\n", eph.isnull().sum()) # conteo
#print(auto.isnull().mean() * 100) # como porcentaje

# No hay duplicados ni missing values

Duplicados: 0

 Missings:
 componente              0
ano4                    0
codusu                  0
nro_hogar               0
Sexo                    0
ch06                    0
ch07                    0
ch08                   17
nivel_ed                0
estado                407
cat_ocup             4109
cat_inac             3278
itf                     0
ipcf                    0
pp07h                4796
pp03c                4526
p47t                    0
p21                     0
pp03g                4264
Edad                    0
adulto_equiv            0
ad_equiv_hogar          0
ingreso_necesario       0
pobre                   0
edad2                   0
ch10                    0
ch12                    0
ch13                    0
ch14                 3465
educ                  142
itf_2025                0
linea_pobreza           0
pp3e_tot             2333
pp3f_tot             2333
ch03                    0
horastrab              30
ix_tot                  0
dtype: int6

In [164]:
### Arreglamos un filtro adicional que encontramos que nos había faltado en la limpieza del TP1


#Estado
def clasificar_estado(estado, ch06):
    if pd.isna(estado) and ch06 < 10:
        return "Menor de 10 años"
    return estado

eph["estado"] = eph.apply(
    lambda row: clasificar_estado(row["estado"], row["ch06"]),
    axis=1)


#cat_ocup
def completar_cat_ocup(cat_ocup, estado):
    if pd.isna(cat_ocup):
        if estado in ["Inactivo", "Menor de 10 años", "Desocupado"]:
            return "No ocupado"
    return cat_ocup

eph["cat_ocup"] = eph.apply(
    lambda row: completar_cat_ocup(row["cat_ocup"], row["estado"]),
    axis=1
)



#Inactivo
def clasificar_estado(cat_inac, estado):
    if pd.isna(cat_inac) and estado == "Ocupado":
        return "Ocupado"
    if pd.isna(cat_inac) and estado == "Desocupado":
        return "Desocupado"
    return cat_inac

eph["cat_inac"] = eph.apply(
    lambda row: clasificar_estado(row["cat_inac"], row["estado"]),
    axis=1)


In [165]:

#dropeamos las filas con nan que son inconsistentes. son muy pocas observacions no afectan los resultados.

eph[["estado","ch08","educ","horastrab"]].isna().sum()

estado         1
ch08          17
educ         142
horastrab     30
dtype: int64

In [166]:
eph = eph.dropna(
    subset=["estado", "ch08", "educ","horastrab"]
)

In [167]:
# Hay valores faltantes?
print("\n Missings:\n", eph.isnull().sum()) # conteo

#Solo quedan nan variables numericas asocidas a personas con trabajo


 Missings:
 componente              0
ano4                    0
codusu                  0
nro_hogar               0
Sexo                    0
ch06                    0
ch07                    0
ch08                    0
nivel_ed                0
estado                  0
cat_ocup                0
cat_inac                0
itf                     0
ipcf                    0
pp07h                4642
pp03c                4389
p47t                    0
p21                     0
pp03g                4127
Edad                    0
adulto_equiv            0
ad_equiv_hogar          0
ingreso_necesario       0
pobre                   0
edad2                   0
ch10                    0
ch12                    0
ch13                    0
ch14                 3439
educ                    0
itf_2025                0
linea_pobreza           0
pp3e_tot             2304
pp3f_tot             2304
ch03                    0
horastrab               0
ix_tot                  0
dtype: int64


# A. Enfoque de validación

In [309]:
#Simplificamos variables categóricas con muchas categorías de manera innecesaria y categorías innecesarias para predicción y nos quedamos con las de interés

eph = eph[
    eph["ch08"] != "Ns./Nr."
]

eph = eph[
    eph["nivel_ed"] != "Ns./Nr."
]

eph = eph[
    eph["cat_inac"] != "Otros"
]


eph["ch10"] = eph["ch10"].astype("category")

eph["cobertura_medica"] = eph["ch08"].apply(
    lambda x: "sin_cobertura_medica" if x == "No paga ni le descuentan" else "con_cobertura_medica"
)



cols = [
    "ano4",
    "componente",
    "codusu",
    "nro_hogar",
    "Sexo",        # corregido
    "ch06",
    "nivel_ed",
    "estado",
    "cat_ocup",    # corregido
    "cat_inac",    # interpretando tu pedido
    "pobre",
    "ch10"
]

eph_variables_seleccionadas = eph[cols]


In [311]:
#Vemos cuales variables eran categóricas

# Variables e información
#print(auto.dtypes)
print(eph_variables_seleccionadas.info())


<class 'pandas.core.frame.DataFrame'>
Index: 7213 entries, 0 to 7339
Data columns (total 12 columns):
 #   Column      Non-Null Count  Dtype   
---  ------      --------------  -----   
 0   ano4        7213 non-null   int64   
 1   componente  7213 non-null   float64 
 2   codusu      7213 non-null   object  
 3   nro_hogar   7213 non-null   int64   
 4   Sexo        7213 non-null   object  
 5   ch06        7213 non-null   int64   
 6   nivel_ed    7213 non-null   object  
 7   estado      6807 non-null   object  
 8   cat_ocup    3231 non-null   object  
 9   cat_inac    3935 non-null   object  
 10  pobre       7213 non-null   int64   
 11  ch10        7213 non-null   category
dtypes: category(1), float64(1), int64(4), object(6)
memory usage: 683.5+ KB
None


In [313]:
#Creamos dummies para las variables categóricas:

eph_con_dummies = pd.get_dummies(
    eph_variables_seleccionadas.drop(columns=["codusu"]),
    drop_first=True,
    dtype=int
)

# Si querés volver a agregar codusu
eph_con_dummies["codusu"] = eph_variables_seleccionadas["codusu"]

eph_con_dummies.head()

,ano4,componente,nro_hogar,ch06,pobre,Sexo_Varón,nivel_ed_Primaria Incompleta (incluye educación especial),nivel_ed_Secundaria Completa,nivel_ed_Secundaria Incompleta,nivel_ed_Sin instrucción,nivel_ed_Superior Universitaria Completa,nivel_ed_Superior Universitaria Incompleta,estado_Inactivo,estado_Menor de 10 años,estado_Ocupado,cat_ocup_Obrero o empleado,cat_ocup_Patrón,cat_ocup_Trabajador familiar sin remuneración,cat_inac_Discapacitado,cat_inac_Estudiante,cat_inac_Jubilado/pensionado,cat_inac_Menor de 6 años,cat_inac_Rentista,ch10_1,ch10_2,ch10_3,codusu
0,2005,1.00,1,46,0,1,0,0,1,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,1,0,125814
1,2005,2.00,1,32,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,125814
2,2005,3.00,1,14,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,1,0,0,125814
3,2005,4.00,1,9,0,1,1,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,1,0,0,125814
4,2005,5.00,1,3,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,1,125814


In [325]:
# Guardo los vectores de variable dependiente y de variable independiente respectivamente:
y = eph_con_dummies['pobre']
X = eph_con_dummies.drop(columns=["pobre", "componente", "nro_hogar", "codusu"])

# Agrego columna de 1s para el intercepto
X = sm.add_constant(X)

### 1. Test de medias

In [333]:
#Test de medias

resultados = []
for col in X.columns:
    x = X[col].values.reshape(-1, 1)
    x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=444, stratify=y)
    t_x = stats.ttest_ind(x_train.flatten(), x_test.flatten())
    resultados.append({
        'Variable': col,
        'N train': x_train.shape[0],
        'Mean train': x_train.mean(),
        'sd train': x_train.std(),
        'N test': x_test.shape[0],
        'Mean test': x_test.mean(),
        'sd test': x_test.std(),
        't-test': t_x.statistic,
        'p-value': t_x.pvalue
    })

estadisticos = pd.DataFrame(resultados)
estadisticos.to_excel('dif_medias_multiple.xlsx', index=False)
estadisticos

C:\Users\admin\anaconda3\Lib\site-packages\scipy\stats\_axis_nan_policy.py:531: RuntimeWarning: Precision loss occurred in moment calculation due to catastrophic cancellation. This occurs when the data are nearly identical. Results may be unreliable.
  res = hypotest_fun_out(*samples, **kwds)


,Variable,N train,Mean train,sd train,N test,Mean test,sd test,t-test,p-value
0,const,5049,1.00,0.00,2164,1.00,0.00,NaN,NaN
1,ano4,5049,"2,016.30",9.91,2164,"2,016.23",9.92,0.28,0.78
2,ch06,5049,34.14,21.59,2164,34.31,21.49,-0.31,0.76
3,Sexo_Varón,5049,0.49,0.50,2164,0.49,0.50,-0.11,0.92
4,nivel_ed_Primaria Incompleta (incluye educació...,5049,0.17,0.38,2164,0.17,0.38,-0.46,0.64
5,nivel_ed_Secundaria Completa,5049,0.20,0.40,2164,0.20,0.40,0.77,0.44
6,nivel_ed_Secundaria Incompleta,5049,0.23,0.42,2164,0.23,0.42,0.12,0.90
7,nivel_ed_Sin instrucción,5049,0.08,0.27,2164,0.06,0.24,1.94,0.05
8,nivel_ed_Superior Universitaria Completa,5049,0.11,0.31,2164,0.11,0.31,-0.26,0.80
9,nivel_ed_Superior Universitaria Incompleta,5049,0.09,0.29,2164,0.11,0.31,-1.43,0.15


### 2. Separación en respondieron_2005 y respondieron_2025

In [343]:
# Separar en dos dataframes según el año
respondieron_2005 = eph[eph['ano4'] == 2005].copy()
respondieron_2005.sample(3)

,ano4,codusu,nro_hogar,ch04,ch06,ch07,ch08,nivel_ed,estado,cat_ocup,cat_inac,itf,ipcf,pp07h,pp03c,p47t,p21,pp03g
16,"2,005.00",224231,2.00,Varón,60,Separado o divorciado,No paga ni le descuentan,Primaria Incompleta (incluye educación especial),Desocupado,Cuenta propia,NaN,0.00,0.00,NaN,NaN,0.00,0.00,NaN
1,"2,005.00",132296,1.00,Mujer,38,Casado,No paga ni le descuentan,Primaria Completa,Inactivo,NaN,Ama de casa,0.00,0.00,NaN,NaN,0.00,0.00,NaN
15,"2,005.00",197933,1.00,Mujer,56,Casado,No paga ni le descuentan,Secundaria Incompleta,Inactivo,NaN,Discapacitado,0.00,0.00,NaN,NaN,0.00,0.00,NaN


In [345]:
respondieron_2025 = eph[eph['ano4'] == 2025].copy()
respondieron_2025.sample(3)

,ano4,codusu,nro_hogar,ch04,ch06,ch07,ch08,nivel_ed,estado,cat_ocup,cat_inac,itf,ipcf,pp07h,pp03c,p47t,p21,pp03g
591,"2,025.00",TQRMNORYYHKMKRCDEGNFJ00860494,1.00,Varón,6,Soltero,Obra social (incluye PAMI),Primaria Incompleta (incluye educación especial),NaN,NaN,Estudiante,0.00,0.00,NaN,NaN,0.00,0.00,NaN
479,"2,025.00",TQRMNOURTHKOLMCDEGNFJ00857204,1.00,Mujer,50,Separado o divorciado,Obra social (incluye PAMI),Superior Universitaria Incompleta,Ocupado,Obrero o empleado,NaN,0.00,0.00,Sí,...un sólo empleo/ocupación/actividad?,"1,350,000.00","1,350,000.00",No
368,"2,025.00",TQRMNOQPPHLOKSCDEFPCH00852522,1.00,Varón,31,Soltero,Obra social (incluye PAMI),Primaria Incompleta (incluye educación especial),Inactivo,NaN,Discapacitado,0.00,0.00,NaN,NaN,0.00,0.00,NaN
